In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from PIL import Image
import time
import os

# Check GPU
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Selected device: {DEVICE}")

if DEVICE.type == 'cuda':
    print(f"GPU : {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()
else:
    print("Error: no GPU")

Device sélectionné : cuda
GPU : NVIDIA Tegra X1


In [10]:
# --- CONFIGURATION ---
MODEL_PATH = "models/resnet18.pth.tar"
NUM_CLASSES = 2
RESOLUTION = (224, 224)

In [3]:
def load_checkpoint(filepath, num_classes):
    print(f"Chargement du checkpoint : {filepath}...")
    
    # Initially load in CPU to save VRAM of GPU during load
    checkpoint = torch.load(filepath, map_location="cpu")
    
    # Get arch name
    arch = checkpoint.get("arch", "resnet18")
    print(f"Detected arch: {arch}")
    
    if arch not in models.__dict__:
        raise ValueError(f"Architecture {arch} not supported by torchvision")
    
    model = models.__dict__[arch]()
    
    # --- RESHAPE of last layer ---
    if hasattr(model, 'fc'):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, num_classes)
    elif hasattr(model, 'classifier'):
        if isinstance(model.classifier, nn.Sequential):
            num_ftrs = model.classifier[-1].in_features
            model.classifier[-1] = nn.Linear(num_ftrs, num_classes)
        else:
            num_ftrs = model.classifier.in_features
            model.classifier = nn.Linear(num_ftrs, num_classes)
    
    # Loading weight
    try:
        model.load_state_dict(checkpoint["state_dict"], strict=True)
        print("Weight successfuly loaded")
    except KeyError:
        model.load_state_dict(checkpoint)
        print("Weight loader with direct format")

    return model.to(DEVICE).eval()

try:
    model = load_checkpoint(MODEL_PATH, NUM_CLASSES)
except Exception as e:
    print(f"Error in loading: {e}")

Chargement du checkpoint : models/resnet18.pth.tar...
Detected arch: resnet18
Weight successfuly loaded


In [4]:
def get_transform(resolution):
    return transforms.Compose([
        transforms.Resize(resolution),
        transforms.CenterCrop(resolution),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                             std=[0.229, 0.224, 0.225]),
    ])

transform = get_transform(RESOLUTION)

In [5]:
def infer_image(model, image_path, transform):
    # Loading picture
    image = Image.open(image_path).convert('RGB')
    input_tensor = transform(image).unsqueeze(0) # Batch of 1
    
    # Send to GPU
    input_tensor = input_tensor.to(DEVICE)
    
    # Inference
    with torch.no_grad(): # Massice save of VRAM
        # Sync before timer
        if DEVICE.type == 'cuda': torch.cuda.synchronize()
        start = time.time()
        
        output = model(input_tensor)
        
        # Sync after timer
        if DEVICE.type == 'cuda': torch.cuda.synchronize()
        end = time.time()
        
        # Probabilities
        probas = F.softmax(output, dim=1)
        
    latency_ms = (end - start) * 1000
    return probas.cpu().squeeze(), latency_ms

In [15]:
IMAGE_PATH = "dataset_capture/27.jpg"
if os.path.exists(IMAGE_PATH):
    print("--- WARMUP (Ignore times) ---")
    _, _ = infer_image(model, IMAGE_PATH, transform)
    print("--- WARMUP FINISH ---\n")

    print(f"Test on : {IMAGE_PATH}")
    probas, latency = infer_image(model, IMAGE_PATH, transform)
    
    # Results
    cible_prob = probas[0].item()
    nocible_prob = probas[1].item()
    
    print(f"Probabilities : [Cible: {cible_prob:.4f}, Nocible: {nocible_prob:.4f}]")
    print(f"Latence Inference : {latency:.2f} ms")
    
    winner = "cible" if cible_prob > nocible_prob else "nocible"
    print(f"Prediction : {winner}")
else:
    print(f"Image {IMAGE_PATH} no found.")

--- WARMUP (Ignore times) ---
--- WARMUP FINISH ---

Test on : dataset_capture/27.jpg
Probabilities : [Cible: 0.7321, Nocible: 0.2679]
Latence Inference : 42.06 ms
Prediction : cible


In [18]:
import glob
import numpy as np

FOLDER_PATH = "dataset_capture"
EXTENSION = "*.jpg"

# Retrieve all .jpg images
search_path = os.path.join(FOLDER_PATH, EXTENSION)
image_paths = glob.glob(search_path)
image_paths.sort() # Ensure deterministic order

if not image_paths:
    print(f"No images found in {FOLDER_PATH}")
else:
    print(f"Found {len(image_paths)} images. Starting batch inference...\n")
    
    # --- WARMUP ---
    # Run one inference to initialize CUDA context and buffers
    if image_paths:
        _ = infer_image(model, image_paths[0], transform)

    latencies = []
    
    # Table Header
    print(f"{'FILENAME':<20} | {'TARGET':<8} | {'NO_TARGET':<9} | {'PREDICTION':<10} | {'LATENCY'}")
    print("-" * 80)

    for img_path in image_paths:
        try:
            # Run Inference
            probas, latency = infer_image(model, img_path, transform)
            latencies.append(latency)
            
            # Parse results (Assuming Index 0 = Target based on your previous snippet)
            prob_target = probas[0].item()
            prob_not_target = probas[1].item()
            
            winner = "TARGET" if prob_target > prob_not_target else "NO_TARGET"
            filename = os.path.basename(img_path)
            
            # Print row
            print(f"{filename:<20} | {prob_target:.4f}   | {prob_not_target:.4f}    | {winner:<10} | {latency:.2f} ms")
            
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

    # --- GLOBAL STATISTICS ---
    print("-" * 80)
    if latencies:
        avg_lat = sum(latencies) / len(latencies)
        # Theoretical Max FPS (excluding disk I/O)
        fps = 1000.0 / avg_lat 
        print(f"Average Inference Latency : {avg_lat:.2f} ms")
        print(f"Theoretical GPU Throughput: {fps:.2f} FPS")

Found 131 images. Starting batch inference...

FILENAME             | TARGET   | NO_TARGET | PREDICTION | LATENCY
--------------------------------------------------------------------------------
10.jpg               | 0.9978   | 0.0022    | TARGET     | 67.20 ms
100.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 53.05 ms
101.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 52.89 ms
102.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 50.35 ms
103.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 33.29 ms
104.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 33.56 ms
105.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 36.14 ms
106.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 36.25 ms
107.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 31.71 ms
108.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 31.50 ms
109.jpg              | 0.0254   | 0.9746    | NO_TARGET  | 31.85 ms
11.jpg               | 0.9418   | 0.0582    | TARGET     